# Get Single RecipCMT Traces.  Using record.py module functions

In [ ]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
    
from scipy import signal

from pyaspect.moment_tensor import MomentTensor
from pyaspect.specfemio.record import Record
import pyaspect.specfemio.record as record
from pyaspect.specfemio.read import _read_headers

## paths

In [ ]:
data_in_dir  = 'data/output/'
data_out_dir = data_in_dir
!ls {data_out_dir}/tmp/TestProjects/CGFR_Test

In [ ]:
projects_fqp = os.path.join(data_out_dir,'tmp','TestProjects','CGFR_Test')
#recip_project_fqp = os.path.join(projects_fqp,'ReciprocalGeometricTestProject') #geometric
#fwd_project_fqp = os.path.join(projects_fqp,'ForwardGeometricTestProject')      #geometric
recip_project_fqp = os.path.join(projects_fqp,'ReciprocalTestProject') #Groningen
fwd_project_fqp = os.path.join(projects_fqp,'ForwardTestProject')      #Groningen
!ls {recip_project_fqp}
print()
!ls {fwd_project_fqp}/run0001/OUTPUT_FILES/plot_source_time_function.txt

## Read Source-Time function

In [ ]:
of_fqp = os.path.join(fwd_project_fqp,'run0001','OUTPUT_FILES')
stf_fqp = os.path.join(of_fqp,'plot_source_time_function.txt')
!ls {stf_fqp}

stf_pair = np.genfromtxt(stf_fqp,dtype=np.float64).transpose()
stf_amp = stf_pair[1,:].copy()
stf_time = stf_pair[0,:].copy()

print(stf_amp)
print(stf_time)

## Read Source-Time function from Reciprocal Force Simulations

In [ ]:
recip_of_fqp = os.path.join(recip_project_fqp,'run0001','OUTPUT_FILES')
recip_stf_fqp = os.path.join(recip_of_fqp,'plot_source_time_function.txt')
!ls {recip_stf_fqp}

recip_stf_pair = np.genfromtxt(recip_stf_fqp,dtype=np.float64).transpose()
recip_stf_amp = recip_stf_pair[1,:].copy()
recip_stf_time = recip_stf_pair[0,:].copy()

print(recip_stf_amp)
print(recip_stf_time)

## Instantiate Record, and test slicing and pandas operations with DataFrames

In [ ]:
recip_record_fqp = os.path.join(recip_project_fqp,'pyheader.project_record')
recip_record = Record(recip_record_fqp)

ne = recip_record.nevents
ns = recip_record.nsrc


print(f'ne:{ne}, ns:{ns}')
print(f'recip: {recip_record.is_reciprocal}')
print(f'Recip Header:\n{recip_record.solutions_df.loc[pd.IndexSlice[:,1],:]}')
print(f'Recip Header:\n{recip_record}')

## create Reciprocal Green's Table (as DataFrame)

In [ ]:
f_low = 1.0
f_high = 100.0
nsamp = 1000
rgf_data_df = record.calc_dataframe_rgf(recip_record,f_low,f_high,nsamp)
rgf_data_df

In [ ]:
#print(rgf_data_df)
#print(rgf_data_df.loc[0,0,0,:,:])
rgf_data_df.loc[(0),:]

## Instantiate Forward Record

In [ ]:
fwd_record_fqp = os.path.join(fwd_project_fqp,'pyheader.project_record')
fwd_record = Record(fwd_record_fqp)
data = fwd_record.data_df
print(data.loc[((1, 0, 0, 0), 'comp_EX')])

## Get Moment tensors to compare with Foward data and also Construct Combinded Reciprocal CMTs. These functions will not be part of record.py module, but make_moment_tensor will be added to utils.py module

In [ ]:
def make_moment_tensor(src_h):
    
    mrr = src_h['mrr']
    mtt = src_h['mtt']
    mpp = src_h['mpp']
    mrt = src_h['mrt']
    mrp = src_h['mrp']
    mtp = src_h['mtp']
    
    h_matrix = np.array([[mrr,mrt,mrp],[mrt,mtt,mtp],[mrp,mtp,mpp]])
    
    return MomentTensor(m_up_south_east=h_matrix)


#print(f'Forward Record Sources:\n{fwd_record_h.solutions_df}')
SrcHeader = fwd_record.solution_cls

d_fwd_src = {}
for eidx, edf in fwd_record.solutions_df.groupby(level='eid'):
    for sidx, sdf in edf.groupby(level='sid'):
        idx = pd.IndexSlice[eidx,sidx]
        src = SrcHeader.from_series(fwd_record.solutions_df.loc[idx])
        #print(src)
        #mag    = src.mw
        #strike = src.strike
        #dip    = src.dip
        #rake   = src.rake
        #mt = MomentTensor(mw=mag,strike=strike,dip=dip,rake=rake)
        mt = make_moment_tensor(src)
        print(mt)
        d_fwd_src[eidx] = mt
        #print(f'mt.aki_m6:\n{mt.aki_richards_m6()}')
        #print(f'header.m6:\n{src.mt}\n')

for key in d_fwd_src:
    print(d_fwd_src[key].m6_up_south_east())

## Construct the Dataframe with the Reciprocal CMT Traces 

In [ ]:
#rgf_cmt_data_df = record.make_cmt_data_from_rgf(rgf_data_df,d_fwd_src,recip_stf_amp,stf_amp)
rgf_cmt_data_df = record.calc_dataframe_composite_recipt_cmt_for_all_events(rgf_data_df,d_fwd_src,recip_stf_amp,stf_amp)

#test_trace = calc_series_composite_recip_cmt_trace(0,0,np.array([1,0,0,0,0,0]),rgf_data_df,recip_stf_amp,stf_amp)
#test_trace['comp_EX']
rgf_cmt_data_df

### Define function for calculation the variance per: [Application to earthquakes...](https://agupubs.onlinelibrary.wiley.com/doi/10.1002/2017JB014230)

In [ ]:
def tensor_variance(t_f, t_r):
    
    sqsum_f  = np.sum(t_f**2)
    sqdifsum = np.sum((t_f - t_r)**2)
    
    return 1. - np.sqrt(sqdifsum/sqsum_f)

## Plot comparison between the Forward and Reciprocal CMT traces

In [ ]:
fig = plt.figure()
irow = 1
icol = 1
iplt = 1

# Filter Forward/CMT traces the same as the Recriprocal traces
sos = signal.butter(3, [f_low,f_high], 'bp', fs=nsamp, output='sos')

sidx = 0
gidx = 0
for eidx in rgf_data_df.index.get_level_values('eid').unique():
    mt_arr = d_fwd_src[eidx].m6_up_south_east()
    for tidx in rgf_data_df.index.get_level_values('trid').unique():
        ic = 0
        trace_df = record.calc_series_composite_recip_cmt_trace(eidx,tidx,mt_arr,rgf_data_df,recip_stf_amp,stf_amp)
        for c in ['comp_EX','comp_NY','comp_Z']: #FIXME with a dictionary
            #r_trace = rgf_cmt_data_df.loc[(eidx,tidx),c]
            r_trace = trace_df[c]
            f_trace = signal.sosfilt(sos,fwd_record.data_df.loc[(eidx,sidx,tidx,gidx),c])
            var = 100*tensor_variance(f_trace,r_trace)
            ax = fig.add_subplot(irow,icol,iplt)
            ax.plot(r_trace,color='black',linestyle='-',linewidth=5,zorder=0,label='Recip',alpha=0.8)
            ax.plot(f_trace,color='gold',linestyle='--',linewidth=2,zorder=1,label='CMT')
            ax.set_title(f'Event:{eidx}, Trace:{tidx}, Comp:{c}, Variance:{var:0.1f}')
            n = len(fig.axes)
            for i in range(n):
                fig.axes[i].change_geometry(n+1, 1, i+1)
            irow = n+1
            iplt = n+1
        ic += 1
                    
 
fig.set_figwidth(15)
fig.set_figheight(3*len(fig.axes))
fig.subplots_adjust(hspace=.35)
plt.show()